In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from textwrap import indent
from IPython.display import Javascript

from liblet import ANTLR, AnnotatedTreeWalker, Tree, pyast2tree, side_by_side

from operator import add, mul, truediv, sub, itemgetter
from fractions import Fraction

from IPython.display import Latex

import importlib

from core.arithmetic_manager import ArithManager
from core.util import *

from core.shunting_yard_parser import ShuntingYardParser
from core.arithmetic_manager import ArithManager
from core.util import *

In [21]:
def solve(expr, domain, show_ast = False, show_blocks = False, show_latex = False):
    AM = ArithManager(domain)
    
    ast = AM.shuntingYardExpr2ast(expr)
    if show_ast: display(ast)
        
    blocks = AM.blocks(ast)
    
    if show_blocks: 
        for block in blocks:
            display(block[1])
        
    MEMORY = dict(blocks)
    main_block = blocks[-1][1]
    child_to_calc = None
    current_block = block_id = None

    while blocks and is_calculable(blocks[0][1]): blocks = blocks[1:]

    while blocks:
        block_id, current_block = blocks[0]
        parent_to_calc = AM.prior(current_block)
        tex = AM.latex(main_block, MEMORY)
        if show_latex: display_latex(tex)
        
        parent_to_calc.children =  [Tree({'type': 'atomExpr', 'value': AM.eval(child, MEMORY), 'priority': 0, '_calc': 'last'}, []) 
                                    if is_next_to_calc(child) else child
                                    for child in parent_to_calc.children]

        if is_calculable(current_block):
            current_block = current_block.children[0]
            blocks = blocks[1:]

        MEMORY[block_id] = current_block
        print(MEMORY)

    tex = AM.latex(main_block, {})
    if show_latex: display_latex(tex)
    
    return main_block.children[0].root['value']





In [22]:
ex1 = '1 + {2 x [(3 + 4 : 2) + 5] x 6} - [ 7 + (1 + 8) : 3 ] x [ 9 : (2 + 1) + 2 ]'
ex2 = '< 2 + 3 > / 4 / < 5 + 6 > / 7'
ex3 = '2 ^ < 3 + 4 ^ 5 : 2 ^ 7 >'
ex4 = '3 + 13 x -<1 + 3>/<2^<1+4> - 6>'
ex5a = '3 / < 4 + 2 > + 2 x { 3 : ( 4 + 5 ) + 2 ^ < 6 x 1/2 > } - < 2 x [ 5 x ( 3 + 4 ) + 5 ] - 1 > / 6 + 1.8'
ex5b = '3 / < 4 + 2 > + 2 x ( 3 : ( 4 + 5 ) + 2 ^ < 6 x 1/2 > ) - < 2 x ( 5 x ( 3 + 4 ) + 5 ) - 1 >'


src = '- (2 + 4) / (3 x 5)'
solve(src, 'Q', show_latex = True)

<IPython.core.display.Latex object>

{'37': ({'type': 'atomExpr', 'value': 6, 'priority': 0, '_calc': 'last'}), '38': ({'type': 'roundBlockExpr', 'priority': 0}: ({'type': 'divProdExpr', 'op': 'x', 'priority': 2}: ({'type': 'atomExpr', 'value': 3, 'priority': 0}), ({'type': 'atomExpr', 'value': 5, 'priority': 0}))), '39': ({'type': 'main'}: ({'type': 'fractExpr', 'priority': 4}: ({'type': 'unaryExpr', 'op': '-', 'priority': 4}: ({'type': 'subExpr', 'ID': '37', 'priority': 0})), ({'type': 'subExpr', 'ID': '38', 'priority': 0})))}


<IPython.core.display.Latex object>

{'37': ({'type': 'atomExpr', 'value': 6, 'priority': 0}), '38': ({'type': 'atomExpr', 'value': 15, 'priority': 0, '_calc': 'last'}), '39': ({'type': 'main'}: ({'type': 'fractExpr', 'priority': 4}: ({'type': 'unaryExpr', 'op': '-', 'priority': 4}: ({'type': 'subExpr', 'ID': '37', 'priority': 0})), ({'type': 'subExpr', 'ID': '38', 'priority': 0})))}


<IPython.core.display.Latex object>

{'37': ({'type': 'atomExpr', 'value': 6, 'priority': 0}), '38': ({'type': 'atomExpr', 'value': 15, 'priority': 0}), '39': ({'type': 'main'}: ({'type': 'fractExpr', 'priority': 3}: ({'type': 'atomExpr', 'value': -6, 'priority': 0, '_calc': 'last'}), ({'type': 'subExpr', 'ID': '38', 'priority': 0})))}


<IPython.core.display.Latex object>

{'37': ({'type': 'atomExpr', 'value': 6, 'priority': 0}), '38': ({'type': 'atomExpr', 'value': 15, 'priority': 0}), '39': ({'type': 'atomExpr', 'value': Fraction(-2, 5), 'priority': 0, '_calc': 'last'})}


<IPython.core.display.Latex object>

Fraction(-2, 5)

In [35]:
import re
token = "-1/5"
if re.match(r'\-?\d+(?:\.\d+)?/\d+(?:\.\d+)?$', token):
    print("UE")

UE
